In [129]:
import os
import time
import csv
import numpy as np
import pandas as pd

import sklearn.metrics
import re
import unicodedata
from tqdm import tqdm
import sys
import hashlib
import random


Input CSV

In [130]:


import re
df = pd.read_csv("../crawled_issue/data_terraform-provider-aws.csv")
df_unique = df.drop_duplicates(subset='Issue ID', keep='first')
df = df_unique
print(df.head())


   Issue ID                                              Title  \
0     35832            Upgrade to go1.21 and make: Fix tidying   
1     35825  [Bug]: LimitExceededException: Certificate has...   
2     35823  [Enhancement]: Enable RoutingConfig argument i...   
3     35822                       td/shield: fix various tests   
4     35821  Fix update operation for authorizer id on apig...   

                                          Issue Body             Closed At  
0  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z  
1  ### Terraform Core Version\n\n1.7.3\n\n### AWS...  2024-02-15T16:43:01Z  
2  ### Description\n\nIn resource `aws_sagemaker_...  2024-02-15T01:32:58Z  
3  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T14:39:44Z  
4  ### Description\r\n\r\nThe change from\r\n```\...  2024-02-15T17:03:14Z  


In [131]:
# !pip3 install openpyxl 
excel_data = pd.read_excel('../dataset/Secret-Regular-Expression.xlsx')

# Read the values of the file in the dataframe
regex = pd.DataFrame(excel_data, columns=[
'Pattern_ID','Secret Type',	'Regular Expression','Source'])

In [132]:

data_dict={}
for j in df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df["Issue Body"][j])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        # file_path = "output.txt"

        # with open(file_path, 'w') as file:
        #                 file.write(cleaned_text)
        data_dict[j] = {'Issue ID':df['Issue ID'][j],'Issue Body':cleaned_text}
        # idx = idx+1
    


cleaned_text_data = pd.DataFrame.from_dict(data_dict, "index")
cleaned_text_data

,Issue ID,Issue Body
0,35832,\r\n### Description\r\n\r\n\r\n\r\n### Relatio...
1,35825,### Terraform Core Version\n\n\n\n### AWS Prov...
2,35823,### Description\n\nIn resource `aws_sagemaker_...
3,35822,\r\n### Description\r\n\r\n\r\n```\r\n------- ...
4,35821,### Description\r\n\r\nThe change from\r\n```\...
...,...,...
31533,5,(Migrating from hashicorp#13246)\r\n\r\nAWS ha...
31534,4,Fixes configuration public ip for spot_instanc...
31535,3,Adds support for tags to `aws_ebs_snapshot` re...
31536,2,testing


Candidate String Generation

In [133]:
###recovered
    
idx = 0
data_dict={}
# start = iter*100000
# end = (iter+1)*100000
for i in regex.index:
    #print(i,regex['Secret Type'][i]) #, regex['Regular Expression'][i])
    # if i%100==0:
    #     print("checkpoint")
    p = re.compile(regex['Regular Expression'][i])
    
    # print("=====================================================================")
    
    for j in df.index:
        
        cleaned_text = cleaned_text_data.loc[j, 'Issue Body']
            # Now you can use 'cleaned_text' for further processing
       
        matches = re.findall(p,cleaned_text)
        for match in set(matches):
                data_dict[idx] = {'Type': regex['Secret Type'][i], 'Issue ID':df['Issue ID'][j],'Candidate String':match} #,'Entropy':shannon_entropy(match)}
                idx = idx+1
    


data = pd.DataFrame.from_dict(data_dict, "index")
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
print(data.shape)
data.to_csv('../crawled_issue/issues-with-candidate-strings.csv')

C:\Users\ahmed\AppData\Local\Temp\ipykernel_3756\3600390447.py:11: FutureWarning: Possible nested set at position 35
  p = re.compile(regex['Regular Expression'][i])
C:\Users\ahmed\AppData\Local\Temp\ipykernel_3756\3600390447.py:11: DeprecationWarning: Flags not at the start of the expression '/(?i)-----\\s*?BEGIN[' (truncated) but at position 1
  p = re.compile(regex['Regular Expression'][i])


(97177, 3)


In [134]:

data = data.rename(columns={'Issue ID': 'Issue_id'})
print(data.shape)
print(data.head())
merged_df = df.merge(data, left_on='Issue ID', right_on='Issue_id')
print(merged_df.shape)
columns_to_remove = ['Issue_id']
merged_df.drop(columns=columns_to_remove, inplace=True)
print(merged_df.columns)

(97177, 3)
           Type  Issue_id   Candidate String
0  Airtable API     28491  key-consolepolicy
1  Airtable API     23663  keyspaces-service
2  Airtable API     21200  keypair-lightsail
3  Airtable API     20665  key-consolepolicy
4  Airtable API     20081  key-group-staging
(97177, 7)
Index(['Issue ID', 'Title', 'Issue Body', 'Closed At', 'Type',
       'Candidate String'],
      dtype='object')


Context Window Generation

In [135]:
def create_context_window(text, target_string, window_size=100):

    target_index = text.find(target_string)
    #print(target_index)

    if target_index != -1:
        start_index = max(0, target_index - window_size)
        end_index = min(len(text), target_index + len(target_string) + window_size)
        context_window = text[start_index:end_index]
        return context_window

    return None

# Apply the create_context_window function to each row in the DataFrame
merged_df['modified_text'] = merged_df.apply(lambda row: create_context_window(row['Issue Body'], row['Candidate String']), axis=1)
print(merged_df.shape)
print(merged_df.head())

(97177, 7)
   Issue ID                                    Title  \
0     35832  Upgrade to go1.21 and make: Fix tidying   
1     35832  Upgrade to go1.21 and make: Fix tidying   
2     35832  Upgrade to go1.21 and make: Fix tidying   
3     35832  Upgrade to go1.21 and make: Fix tidying   
4     35832  Upgrade to go1.21 and make: Fix tidying   

                                          Issue Body             Closed At  \
0  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z   
1  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z   
2  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z   
3  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z   
4  <!---\r\nSee what makes a good Pull Request at...  2024-02-15T17:58:18Z   

              Type                          Candidate String  \
0  Generic Pattern            TestAccECSTaskDefinition_basic   
1  Generic Pattern  TestAccSecretsManagerSecret_basicRe

In [136]:
count=0
inverse_count=0

for i in range(merged_df.shape[0]):
  #print(i)
  main_string=merged_df['Issue Body'][i]
  substring=merged_df['Candidate String'][i]
  #print(main_string.find(substring))
  if main_string.find(substring)!=-1:
    count+=1
  else:
    inverse_count+=1
print("Mismatch: "+str(inverse_count))
print("Ok: "+str(count))

Mismatch: 489
Ok: 96688


In [137]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

In [138]:
X_issue_ids = merged_df['Issue ID'].tolist()
X_text_test = merged_df['Issue Body'].tolist()  # Convert the 'text' column to a list of strings
X_candidate_test = merged_df['Candidate String'].tolist()  # Convert the 'candidate_string' column to a list of strings

In [139]:
def encode_texts(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    return encodings

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [140]:
# Define a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, text_encodings, candidate_encodings, labels):
        self.text_encodings = text_encodings
        self.candidate_encodings = candidate_encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): #it works fine for training
        text_input_ids = self.text_encodings['input_ids'][idx]
        text_attention_mask = self.text_encodings['attention_mask'][idx]
        candidate_input_ids = self.candidate_encodings['input_ids'][idx]
        candidate_attention_mask = self.candidate_encodings['attention_mask'][idx]
        label = torch.tensor(self.labels[idx])

        return text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, label

In [141]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model_path = "../models/rmsprop_cntxt100_data25k_pre.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode for inference

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [142]:
text_body_encodings_test = encode_texts(X_text_test)
candidate_encodings_test = encode_texts(X_candidate_test)

In [ ]:
print(len(X_text_test))
Y_labels = [0] * len(X_text_test)
Y = np.array(Y_labels)
Y_ =Y.astype(int)
print(Y_)

In [ ]:
test_dataset = CustomDataset(text_body_encodings_test, candidate_encodings_test, Y_)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
model.to(device)
model.eval()  # Set the model in evaluation mode
c=0
predicted_labels_list = []
with torch.no_grad():

    for batch in test_loader:
        print("Batch %d"%c)
        c+=1

        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = batch

        # Move tensors to the device
        text_input_ids, text_attention_mask, candidate_input_ids, candidate_attention_mask, labels = (
            text_input_ids.to(device),
            text_attention_mask.to(device),
            candidate_input_ids.to(device),
            candidate_attention_mask.to(device),
            labels.to(device)
        )

        # Perform inference

        outputs = model(input_ids=text_input_ids.type(torch.LongTensor).cuda(), attention_mask=text_attention_mask.type(torch.LongTensor).cuda())
        predicted_labels = torch.argmax(outputs.logits, dim=1)

        # print(f"predicted_labels: {predicted_labels}")
        predicted_labels_list.append(predicted_labels[0])

In [ ]:
predicted_labels_list_output = [f.cpu().numpy().tolist() for f in predicted_labels_list]
print(predicted_labels_list_output)

Flagged CSV Generation

In [ ]:
repolink="https://github.com/hashicorp/terraform-provider-aws"
data_res={}
indx=0
for i in range(len(predicted_labels_list_output)):
    if predicted_labels_list_output[i] == 1 :
        data_res[indx]={"Issue ID":df[df['Issue ID'] == X_issue_ids[i]]['Issue ID'].values[0],"Issue Body":df[df['Issue ID'] == X_issue_ids[i]]['Issue Body'].values[0],"Candidate String":X_candidate_test[i],"Repolink":repolink}
        indx=indx+1
if(indx==0):
    data_res[indx]={"Issue ID":"N/A","Issue Body":"N/A","Candidate String":"N/A","Repolink":repolink}

print(data_res)
data = pd.DataFrame.from_dict(data_res, "index")
data=data.drop_duplicates(subset=["Issue ID", "Candidate String"], keep='first')
print(data.shape)
data.to_csv('../results/flagged_terraform-provider-aws.csv')
        

In [ ]:

secret_indexes = []
for i in range(len(predicted_labels_list_output)):
    if predicted_labels_list_output[i] == 1 :
        secret_indexes.append(X_issue_ids[i])
secret_indexes = list(set(secret_indexes))
print(secret_indexes)
print(len(secret_indexes))